In [ ]:
import datetime
import re
import pandas as pd
import numpy as np


path_311 = "data/311_Service_Requests_from_2010_to_Present.csv"
path_nta = "data/Neighborhood Tabulation Areas.geojson"
path_nta_population = "data/New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv"

In [ ]:
df_311 = pd.read_csv(path_311)

In [ ]:
df_311.columns

Make sure that "Unique Key" column can be an index

In [ ]:
df_311["Unique Key"].is_unique

In [ ]:
df_311.set_index("Unique Key");

Check what complaint types do not have "Location" set. If there would be few of those or the categories would be of no interest to us, we would have dropped them.

In [ ]:
df_311[df_311["Location"].isna()].groupby("Complaint Type").count()["Unique Key"].sort_values()[-20:]

We drop the State Plane coordinates as they are just a different format of the coordinates present in Latitude and Longitude.

In [ ]:
df_311.drop(columns=["X Coordinate (State Plane)", "Y Coordinate (State Plane)"], inplace=True)

We drop the "Location" column as it is just the Latitude and Longitude columns combined.

In [ ]:
df_311.drop(columns="Location", inplace=True)

# After dropping Location and X/Y Coordinates in State Plane 

Checking the complaint types of the entries with "Bridge Highway Segment" present.

In [ ]:
hw_segment_present = df_311[~df_311["Bridge Highway Segment"].isna()]

In [ ]:
hw_segment_present["Complaint Type"].unique()

In [ ]:
df_311["Complaint Type"].value_counts(ascending=True)

We realize that some of the complaint types are incorrct. Many of such entries categories appear only once in the dataset. We drop those entries as we cannot interpret such complaints.

In [ ]:
invalid_complaints = list(df_311["Complaint Type"].value_counts(ascending=True)[
    df_311["Complaint Type"].value_counts(ascending=True) == 1].index)
df_311 = df_311[~df_311["Complaint Type"].isin(invalid_complaints)]

## Casting fields to datetime

In [ ]:
unique_created_dates = df_311["Created Date"].unique()

In [ ]:
unique_closed_dates = df_311["Closed Date"].unique()

In [ ]:
unique_closed_dates = list(pd.DataFrame(unique_closed_dates)[0].dropna())

We will verify if the fields with dates have proper formatting. This still does not guarantee that they are logically correct, that is in some expected range, but tells us whether or not we would be able to parse them.

In [ ]:
txt='01/17/2026 10:12:35 PM'

# MMDDYYYY 1
re1='((?:[0]?[1-9]|[1][012])[-:\\/.](?:(?:[0-2]?\\d{1})|(?:[3][01]{1}))[-:\\/.](?:(?:[1]{1}\\d{1}\\d{1}\\d{1})|(?:[2]{1}\\d{3})))(?![\\d])'
# White Space 1
re2='( )'	
# HourMinuteSec
re3='((?:(?:[0-1][0-9])|(?:[2][0-3])|(?:[0-9])):(?:[0-5][0-9])(?::[0-5][0-9])?(?:\\s?(?:am|AM|pm|PM))?)'

rg = re.compile(re1+re2+re3,re.IGNORECASE|re.DOTALL)

def try_matching(txt):
    m = rg.search(str(txt))
    if not m:
        print("NOT A MATCH!")
        print(txt)
        return 1
    return 0

In [ ]:
mapped_created = list(map(lambda x: try_matching(x), unique_created_dates))
sum(mapped_created)

In [ ]:
mapped_closed = list(map(lambda x: try_matching(x), unique_closed_dates))
sum(mapped_closed)

There is one entry among closed dates that is in the third millenium. We assume time travel is impossible, so the complaint couldn't have been closed in the future. Dropping the entry with it.

In [ ]:
df_311 = df_311[df_311["Closed Date"] != "03/30/3027 12:00:00 AM"]

Parsing the dates and casting them to pandas datetime type.

In [ ]:
df_311["Created Date"] = pd.to_datetime(df_311["Created Date"])

In [ ]:
df_311["Closed Date"] = pd.to_datetime(df_311["Closed Date"])

In [ ]:
df_311["Created Date"].hist()

We note that the "Created Date" column appears to contain only dates that are possible (within reasonable range. Moreover, we clearly see an increase in the number of complaints overall since the year 2012. This might be either due to growing popularity of 311, increase in population or worsening conditions in the city. It might be a combination of those factors, so we do not draw final conclusions yet.

In [ ]:
df_311["Closed Date"].hist()

Clearly there is something wrong with the values of ticket closed date. There are entries that have the dates set in 1900s:

In [ ]:
df_311[df_311["Closed Date"] < datetime.datetime(2009, 1, 1)].head(2)

We assume that those which do not have "Status" set to closed should not have the "Closed Date" set in the first place as the only other status present among them is "Pending". Thus we set their "Closed Date" to NaN. We remove the entries which had the status closed and "Closed date" set before 2010.

In [ ]:
df_311[(df_311["Closed Date"] < datetime.datetime(2010, 1, 1)) & (df_311["Status"] == "Closed")]["Unique Key"].count()

In [ ]:
df_311.drop(df_311[(df_311["Closed Date"] < datetime.datetime(2010, 1, 1)) & (df_311["Status"] == "Closed")].index, inplace=True)

One of the issues we have spotted is the fact that there are entries with "Closed Date" before "Created Date". We assume this might be a way of dealing with complaints submitted for the problems that were already resolved. Those may be also plain mistakes. We decide to drop all such rows.

In [ ]:
df_311.drop(df_311[df_311["Closed Date"] < df_311["Created Date"]].index, inplace=True)

In [ ]:
df_311["Closed Date"].hist()

We also decide to remove the rows that have "Closed Date" after today. This is because one expects the tickets with "Closed Date" present to be actually closed already. There is few such cases, so it should not pose a problem.

In [ ]:
df_311[df_311["Closed Date"] > datetime.datetime.today()]["Unique Key"].count()

In [ ]:
df_311.drop(df_311[df_311["Closed Date"] > datetime.datetime.today()].index, inplace=True)

There are two more columns with dates in them:

In [ ]:
df_311["Due Date"] = pd.to_datetime(df_311["Due Date"])

In [ ]:
df_311["Due Date"].hist()

We assume that the Due Dates set in 1900s are incorrect and replace them by NaNs.

In [ ]:
df_311[df_311["Due Date"] < datetime.datetime(2010, 1, 1)]

In [ ]:
df_311["Due Date"] = df_311["Due Date"].replace(datetime.datetime(1900, 1, 2), np.NaN)

In [ ]:
df_311["Due Date"].hist()

In [ ]:
unique_resolution_dates = df_311["Resolution Action Updated Date"].unique()

In [ ]:
sum(map(lambda x: try_matching(x), unique_resolution_dates))

In [ ]:
dates_without_incorrect = df_311[~df_311["Resolution Action Updated Date"].isin(incorrect_dates)]

In [ ]:
dates_without_incorrect["Resolution Action Updated Date"].unique()

In [ ]:
incorrect_dates = ["10/30/2926 11:51:00 AM", "05/25/2510 10:15:00 AM", "03/06/2927 12:30:00 PM"]

In [ ]:
df_311["Resolution Action Updated Date"] = df_311[~df_311["Resolution Action Updated Date"].isin(incorrect_dates)]["Resolution Action Updated Date"]
df_311["Resolution Action Updated Date"] = pd.to_datetime(df_311["Resolution Action Updated Date"])

In [ ]:
df_311["Resolution Action Updated Date"].hist()

We remove the rows where the Resolution Action Updated Date is in the future. By definition it should be the date when agency has last updated the entry.

In [ ]:
df_311.drop(df_311[df_311["Resolution Action Updated Date"] > datetime.datetime.today()].index, inplace=True)

In [ ]:
df_311["Resolution Action Updated Date"].hist()

We decide to drop the entries with "Resolution Action Updated Date" before 2010 as that is the date where the data has started being collected.

In [ ]:
df_311.drop(df_311[df_311["Resolution Action Updated Date"] < datetime.datetime(2010, 1, 1)].index, inplace=True)

In [ ]:
df_311["Resolution Action Updated Date"].hist()

## Casting to "category" type

We note that many of the columns can take only one of a small set of possible values. We cast them to "category" type.

In [ ]:
category_columns = ["Agency", "Agency Name", "Complaint Type", "Descriptor", "Location Type",
                    "Community Board", "Address Type", "City", "Landmark", "Facility Type", "Status",
                    "Resolution Description", "Borough", "Open Data Channel Type", "Park Facility Name",
                    "Park Borough", "Vehicle Type", "Taxi Company Borough", "Bridge Highway Direction", "Road Ramp"]

In [ ]:
df_311[category_columns] = df_311[category_columns].astype("category")

In [ ]:
df_311.to_pickle("data/311_Service_Requests_from_2010_to_Present_small.pkl")

# Wordcloud

In [ ]:
# import pandas as pd
# df_311 = pd.read_pickle("data/311_Service_Requests_from_2010_to_Present_small.pkl")

We will generate wordcloud from Complaint Type, Descriptor, and Resolution Descriptor. Those can be narrowed down later.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def create_wordcloud(pandas_series, max_words=2000, max_font_size=None, mask=None, word_level=False, filepath=None):
    wordcloud = WordCloud(
        width=3000,
        height=2000,
        max_words=max_words,
        max_font_size=max_font_size,
        background_color='white',
        stopwords=STOPWORDS,
        random_state=1,
        mask=mask,
        contour_width=3,
        contour_color='black'
    )
    
    text = pandas_series.astype(str).values
    processed_text = wordcloud.process_text(" ".join(text))
    wordcloud.generate_from_frequencies(processed_text)
    fig = plt.figure(
        figsize = (40, 30),
        facecolor = 'k',
        edgecolor = 'k')
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.tight_layout(pad=0)
    if filepath:
        wordcloud.to_file(filepath)

In [ ]:
nyc_mask = np.array(Image.open("data/NYC_silhouette.png"))
nyc_mask[nyc_mask > 0] = 255

In [ ]:
create_wordcloud(df_311["Complaint Type"], mask=nyc_mask, filepath="data/complaint_type_wc.png")

In [ ]:
create_wordcloud(df_311["Resolution Description"], filepath="data/resolution_description_wc.png")

In [ ]:
create_wordcloud(df_311["Descriptor"], filepath="data/resolution_descriptor_wc.png")

# Investigating how agencies are split to take care of smaller regions in NYC

In [ ]:
# import pandas as pd
# df_311 = pd.read_pickle("data/311_Service_Requests_from_2010_to_Present_small.pkl")

In [ ]:
school_agencies = [agency for agency in list(df_311["Agency Name"].unique()) if "School - " in agency]
len(school_agencies)

In [ ]:
df_311[df_311["Agency Name"].isin(school_agencies)]["Agency"].unique()

Many of the agencies are schools. All of them are mapped to agency "DOE". Schools are divided into NYC school districts. The map of such districts is available https://data.cityofnewyork.us/Education/School-Districts/r8nu-ymqj , but we don't see a dataset that could map 3-1-1 Agency Names to school districts.

In [ ]:
not_school_agencies = [agency for agency in list(df_311["Agency Name"].unique()) if not "School - " in agency]
len(not_school_agencies)

In [ ]:
df_311[df_311["Agency"] == "NYPD"]["Agency Name"].unique()

We see that a single "Agency" entry maps to multiple "Agency Name" entries that in some cases are equivallent, such as "NYPD" and "New York City Police Department". This indicates that we should first look at the "Agencies" themselves as "Agency Names" may not be meaningful.

In [ ]:
list(df_311["Agency"].unique())

NYPD is split into Police Precints: https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz

In [ ]:
list(df_311[df_311["Agency"] == "OMB"]["Agency Name"].unique())

DSNY (Department of Sanitation) districts: https://data.cityofnewyork.us/City-Government/DSNY-Districts/6j86-5s7z

FDNY (Fire Division). Battallions: https://data.cityofnewyork.us/Public-Safety/Fire-Battalions/uh7r-6nya , divisions (coarse grained): https://data.cityofnewyork.us/Public-Safety/Fire-Divisions/hkpx-aaxc

Overall, we have found geographical subdivisions for DOE, NYPD, DSNY and FDNY.

# Louis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
SAMPLE_PATH = "data/extract.csv"
path_nta = "data/Neighborhood Tabulation Areas.geojson"
path_nta_population = "data/New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv"

In [ ]:
df = pd.read_csv(SAMPLE_PATH)

In [ ]:
df.columns

In [ ]:
df.shape

We check that the Unique Key truly is unique, and that no NaN's appear in the column.

In [ ]:
assert df["Unique Key"].is_unique
assert not df["Unique Key"].isnull().values.any()

In [ ]:
for c in df.columns:
    if "Date" in c:
        print("Converting", c)
        df[c] = pd.to_datetime(df["Created Date"])

In [ ]:
df["Created Day"] = df["Created Date"].apply(lambda d: d.day)
df["Created Month"] = df["Created Date"].apply(lambda d: d.month)
df["Created Year"] = df["Created Date"].apply(lambda d: d.year)

In [ ]:
def heatbarplot(value_counts):
    max_v = value_counts.max()
    min_v = value_counts.min()
    clrs = {k:np.array([(v-min_v)/(max_v-min_v), 0., 0.]) for k, v in zip(value_counts.keys(), value_counts.values)}
    sns.barplot(x=value_counts.keys(), y=value_counts.values, palette=clrs)

In [ ]:
heatbarplot(df["Created Month"].value_counts())

In [ ]:
heatbarplot(df["Created Year"].value_counts())

In [ ]:
heatbarplot(df["Created Day"].value_counts())

In [ ]:
df["Agency"].value_counts().plot.bar()

In [ ]:
ct_vcs = df["Complaint Type"].value_counts()
pd.set_option('display.max_rows', 1000)
ct_vcs

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:

ct_vcs.head().plot.bar()

In [ ]:
import folium


In [ ]:
def plotpts(geodf, color="#ff0000", popup="None"):
    clean = geodf[geodf["Latitude"].notnull()]
    clean = clean[clean["Longitude"].notnull()]
    m = folium.Map(location=[clean["Latitude"].values[0], clean["Longitude"].values[0]], zoom_start=10)
    #for c in zip(clean["Latitude"].values, clean["Longitude"].values):
    for i, row in clean.iterrows():
        folium.CircleMarker(location=(row["Latitude"], row["Longitude"]),
                            radius= 1,
                            color=color(row) if callable(color) else color,
                            popup=popup(row) if callable(popup) else popup,
                            fill=True).add_to(m)
    return m
plotpts(df[df["Complaint Type"] == "Noise - Residential"])

In [ ]:
cfun = lambda row: "#aa3300" if row["Complaint Type"] == "Noise - Residential" else \
"#00aa33" if row["Complaint Type"] == "Street Condition" else \
"#aa0066" if row["Complaint Type"] == "Street Light Condition" else \
"#cccc00" if row["Complaint Type"] == "HEAT/HOT WATER" else \
"#3300aa" if row["Complaint Type"] == "Illegal Parking" else "#444444"
popup = lambda row: str(popup)
df2k = df.sample(n=2000, random_state=1)
plotpts(df2k, color=cfun, popup=popup)

In [ ]:
print("Out of", df.shape[0], "entries:")
facility_df = df[df["Facility Type"].notnull()]
print(facility_df.shape[0], "have associated facility type.")
facility_df["Facility Type"].value_counts()

We have 3 disctinct associated facility type. Mostly Precinct, then DSNY Garage then a few School.

In [ ]:
df["Open Data Channel Type"].value_counts()

In [ ]:
df["Status"].value_counts()

In [ ]:
df["Borough"].value_counts()

In [ ]:
df2k = df.sample(n=2000, random_state=1)

In [ ]:
cfun = lambda row: "#00cc00" if row["Status"] == "Closed" else \
"#cccc00" if row["Status"] == "Pending" else \
"#cc0000" if row["Status"] == "Open" else \
"#0000cc" if row["Status"] == "Assigned" else "#444444"
popup = lambda row: str(row)
plotpts(df2k, color=cfun, popup=popup)

Lead: analyse precinct efficiency, "time to solve", "solving rate"

In [ ]:
df["Vehicle Type"].value_counts()

In [ ]:
df["Descriptor"].describe()

In [ ]:
df["Descriptor"].value_counts()

Neighborhood rating system, show subscores for:

- noise disturbance (banging/pounding, loud party etc.)
- infrastructure condition (pothole descriptor, road condition/street light condition etc.)
- private residence condition (rat sighting, vermin, mold, etc.)
- safety (mix with 911?)

and create a general index too. mix with property prices to find overvalued/undervalued properties with predictive model

### Part 2: Complaint type classification

We manually classified complaint types into the following categories to use at a later point:


In [ ]:
import json

with open("complaint_type_cls_legend.json") as fp_legend:
    ct_legend = json.load(fp_legend)
ct_legend   

The following classification was performed:

In [ ]:
with open("complaint_type_cls.json") as fp_data:
    ct_data = json.load(fp_data)
ct_data

In [ ]:
df["Complaint Type Class Index"] = df["Complaint Type"].apply(lambda k: ct_data[k])
ct_legend_inv = {v:k for k, v in ct_legend.items()}
df["Complaint Type Class"] = df["Complaint Type Class Index"].apply(lambda k: ct_legend_inv[k])
df

In [ ]:
df["Complaint Type Class"].value_counts().plot.bar()

### Part 3: Plotting GeoJSON NTAs with some data

In [ ]:
import folium
import json

m = folium.Map(location=(40.730610, -73.935242), zoom_start=11)

with open("data/Neighborhood Tabulation Areas.geojson") as fp:
    geojson = json.load(fp)
    
for feature in geojson["features"]:
    #print(feature["properties"]["ntacode"])
    gj = folium.GeoJson(feature, name=feature["properties"]["ntaname"],
                       style_function=lambda feature: {
        'fillColor': "#00ff00",
        'color' : "#ff0000",
        'weight' : 1,
        'fillOpacity' : 0.5,
        })
    folium.Popup(feature["properties"]["ntaname"] + '\n' + feature["properties"]["ntacode"]).add_to(gj)
    gj.add_to(m)

m

In [ ]:

PT_1 = (40.627, -73.966)
PT_2 = (40.627, -73.986)

m = folium.Map(location=(40.730610, -73.935242), zoom_start=11)

with open("data/Neighborhood Tabulation Areas.geojson") as fp:
    geojson = json.load(fp)
    
for feature in geojson["features"]:
    #print(feature["properties"]["ntacode"])
    gj = folium.GeoJson(feature, name=feature["properties"]["ntaname"],
                       style_function=lambda feature: {
        'fillColor': "#00ff00",
        'color' : "#ff0000",
        'weight' : 1,
        'fillOpacity' : 0.5,
        })
    folium.Popup(feature["properties"]["ntaname"] + '\n' + feature["properties"]["ntacode"]).add_to(gj)
    gj.add_to(m)
    break

folium.Marker(PT_1).add_to(m)
folium.Marker(PT_2).add_to(m)

m

In [ ]:
import shapely.geometry as sg

geometry = geojson["features"][0]["geometry"]
pt_1 = sg.Point(PT_1)
pt_2 = sg.Point(PT_2)
polygon = sg.shape(geometry)
print(polygon.contains(pt_1))
print(polygon.contains(pt_2))

# Olivier

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import pickle as pk
import seaborn as sns
import plotly.graph_objects as go

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
with open("data/311_Service_Requests_from_2010_to_Present_small.pkl", 'rb') as f:
    df = pk.load(f)

In [ ]:
df['Complaint Type'].cat.remove_unused_categories(inplace=True)
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'])
df = df.assign(Quantity=pd.Series(np.ones(len(df.index))).values)

# Seasonality

In [ ]:
def plot_season(col="Complaint Type", num_labels=30, plotting="plt", frequency="M", date_grouper="Created Date"):
    if plotting not in ["plt", "plotly"]:
        raise NotImplementedError("Invalid plotting method")
    ### compute complaint types with most occurences
    most_freq_labels = [x for x in df[col].value_counts().keys().values[:num_labels]]

    ### group requests by month
    season = df.groupby([col, pd.Grouper(key=date_grouper, freq=frequency)])['Quantity']\
        .sum().reset_index().sort_values(date_grouper)
    season.fillna({'Quantity': 0}, inplace=True)


    ### get corresponding times and quantities 
    times = [season[season[col] == c][date_grouper] for c in most_freq_labels]
    qts = [season[season[col] == c]['Quantity'] for c in most_freq_labels]

    ### plot
    if plotting.lower() == "plt":
        plt.figure(figsize=(16,8))
        pal = sns.color_palette("Set1")
        plt.stackplot(times[0], qts, labels=most_freq_labels, colors=pal)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.tight_layout()
        plt.xlabel("Time")
        plt.ylabel("Count of requests")
        plt.title(col)
        plt.show();
    elif plotting.lower() == "plotly":
        fig = go.Figure({"layout": {"title": {"text": col}}})
        for lab, q, t in zip(most_freq_labels, qts, times):
            fig.add_trace(go.Scatter(
                x=t, y=q,
                hoverinfo='x+y',
                mode='lines',
                name=lab,
                stackgroup='one' # define stack group
            ))
        fig.show()
    else:
        raise NotImplementedError("Woups")

In [ ]:
plot_season('Complaint Type', 100, plotting="plotly")

In [ ]:
plot_season("Agency", 20, plotting="plotly")

In [ ]:
plot_season("Descriptor", 30, plotting="plotly")

## Observations
Around March-April 2014, no more _Heating_ entries are registered, but _HEAT/HOT WATER_ start to appear. It's probable it was replaced.

Heating problems are obviously more reported during winter, peaking between December and January of each year.

Similarly, _Street Conditions_ are more reported in spring, around March. 

Noise knows 2 peaks and 2 pits: the heart of summer and winter seem calmer. Probably because less people are in the street, being mostly on holiday (or inside, during winter). Meanwhile spring and autumn have more people around, creating more nuisance.

Interestingly, while most of requests are gradually growing (due to the service being more and more popular), some manage to keep a steady level, notably Sewers.

Sewers have another funny feature: August 2011 raises from the usual 2-3k monthly requests to 8k requests. It is possible that the reason for that is a light earthquake, that happend August 22nd, possibly disrupting the sewers system.

# Julien

In [ ]:
import pandas as pd

path_311 = "data/extract.csv"
path_nta = "data/Neighborhood Tabulation Areas.geojson"
path_nta_population = "data/New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv"

In [ ]:
df_311 = pd.read_csv(path_311)
df_nta_population = pd.read_csv(path_nta_population)

Let's take a look at the unique columns:

In [ ]:
df_311.columns

Some columns are specific to certain complaint types: **Landmark**, **Park Facility Name**, **Vehicle Type**, **Taxi Company Borough**, **Taxi Pick Up Location**, **Bridge Highway Name**, **Bridge Highway Direction**, **Road Ramp**, **Bridge Highway Segment**. If those fields are empty for most complaints, we might as well drop the corresponding columns.

In [ ]:
df_311

### Complaints

In [ ]:
df_311["Complaint Type"].describe()

In [ ]:
complaints = df_311["Complaint Type"].unique().tolist()
complaints.sort()
print(complaints)

One entry has to be removed: './validate_form.php'. Should check for other outliers in the full dataset.

In [ ]:
complaint_count = df_311.groupby("Complaint Type") \
                        .count()["Unique Key"] \
                        .sort_values(ascending=False) 
      
complaint_count.plot.barh(x="Complaint Type", y="Unique Key", logx=True, figsize=(15, 40));

Let's look at the top 30 complaints:

In [ ]:
complaint_count[:30]

As we can see, some complaints are redundant and should be grouped. (HEATING, PAINT/PLASTER)

### Timestamps

In [ ]:
df_311["Created Date"] = pd.to_datetime(df_311["Created Date"])
df_311["Closed Date"] = pd.to_datetime(df_311["Closed Date"])

In [ ]:
df_311

In [ ]:
years = df_311["Created Date"].apply(lambda d: d.year).rename("cyear")

In [ ]:
years

In [ ]:
years.groupby(years) \
     .count() \
     .sort_values(ascending=False) \
     .plot.barh(x="cyear", y="Count");

Values were randomly sampled so there is a margin of error, and 2019 is not over, but it seems like there is a linear increase in the number of 311 requests - the service gets more popular.

### Location

In [ ]:
df_311[df_311["Location"].isna()].iloc[0]

In [ ]:
df_311["Incident Zip"].describe()

In [ ]:
df_311[df_311["Incident Zip"].isna()].iloc[0]

Depending on the type of complaint, location might be undefined (traffic light at intersection, literature request).

In [ ]:
df_311[~(df_311["Incident Zip"].isna())]

In [ ]:
df_311[~(df_311["Longitude"].isna())]

We do have the exact location for most of the requests (>80%).

Note that **X Coordinate (State Plane)** / **Y Coordinate (State Plane)** might be more precise than **Longitude** / **Latitude**.

### Request origin

In [ ]:
channel_count = df_311.groupby("Open Data Channel Type") \
                        .count()["Unique Key"] \
                        .sort_values(ascending=False) 
      
channel_count.plot.barh(x="Channel", y="Count");

Most requests are still submitted through a phone call.

### Focus on certain complaints

Let's look again at the top 30 complaints:

In [ ]:
complaint_count[:30]

Things that stand out: 
- Noise most recurrent complaint, different types of noise
- Followed by Water / Heating systems / Plumbing / Sewer
- Things that refer to the bad condition of something: Street Condition, Street Light Condition, Traffic Signal Condition, Unsanitary conditions, Dirty Conditions, Sanitation Condition, Sidewalk Condition and many more

### Analysis ideas

List of things that could be done:
- Per Neighborhood Tabulation Area (NTA): Top 30 Problems, frequency w.r.t the size of the NTA, look into trends
- Average time to resolution over the years (compare NTAs, agencies)
- Word cloud per NTA
- How well prepared for / resistent to external events, such as blizzards

Problem: missing something interesting, that goes beyond the simple analysis. Make use of another dataset? Do something for social good: either help agencies fix their problems more efficiently (redraw borders of NTAs, encourage communication between departments), or give people some advice (Fix your heating system before the winter, because there is a high probability of failure - Do you really want to move to that neighborhood? It has a couple of problems, here is a better one, which is much more silent). 

# Add NTA information to complaints

In [ ]:
%matplotlib inline

import json
import math
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import shape, Point

In [ ]:
# load GeoJSON file containing NTAs
with open(path_nta) as f:
    js = json.load(f)

def get_nta(location_str):
    """Retrieves the NTA of a given location in NYC.
    Args:
        location_str (String): location associated to the complaint, that is '(latitude, longitude)'
    Returns:
        String: code of the NTA the location is part of, or NaN if the location is not within an NTA
    """
    location = eval(location_str)
    latitude = location[0]
    longitude = location[1]
    point = Point(longitude, latitude)
    for feature in js["features"]:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            nta_code = feature["properties"]["ntacode"]
            return nta_code
    return np.nan
    
get_nta('(40.77382381576739,-73.95411117806607)')

In [ ]:
df_311["NTA"] = np.nan
df_311.loc[df_311.Location.notnull(), "NTA"] = df_311.Location[df_311.Location.notnull()].apply(get_nta)

In [ ]:
print(f"Added the NTA to {df_311.NTA.notnull().sum()} complaints")

In [ ]:
no_loc_complaints = df_311[df_311.NTA.isnull()]
complaint_count = no_loc_complaints.groupby("Complaint Type") \
                                   .count()["Unique Key"] \
                                   .sort_values(ascending=False) 
      
complaint_count.head(10).plot.barh(x="Complaint Type", y="Count", figsize=(15, 10));

**Street Lights** / **Traffic Signal** are often at intersections, which don't have an exact address. We also have complaints which don't refer to a specific location such as **Benefit Card Replacement**. We might have to watch out for **Street Light Condition** complaints, because they also are among the most frequent complaints in the raw dataset.

# Analyse noise based on time of the day

## Filter noise complaints

In [ ]:
complaints_lowercase = list(map(lambda x: (x, x.lower()), complaints))
noise_complaint_types = [complaint[0] for complaint in complaints_lowercase if "noise" in complaint[1]]
print(noise_complaint_types)

In [ ]:
noise_complaints = df_311[df_311["Complaint Type"].isin(noise_complaint_types)]
complaint_count = noise_complaints.groupby("Complaint Type") \
                                  .count()["Unique Key"] \
                                  .sort_values(ascending=False) 
      
complaint_count.plot.barh(x="Complaint Type", y="Unique Key", logx=True, figsize=(15, 10));

In [ ]:
sorted_requests = "data/311_Service_Requests_from_2010_to_Present_sorted.csv"

In [ ]:
from datetime import datetime

start = datetime(2019, 8, 5)
end = datetime(2019, 10, 21)

In [ ]:
names = df_311.columns.tolist()[:-1]
sorted_df = pd.read_csv(sorted_requests, header=None, names=names, nrows=500000)
sorted_df = sorted_df[sorted_df["Complaint Type"].isin(noise_complaint_types)]
sorted_df["Created Date"] = pd.to_datetime(sorted_df["Created Date"])
sorted_df = sorted_df[(start <= sorted_df["Created Date"]) & (sorted_df["Created Date"] < end)]

In [ ]:
sorted_df["Created Day"] = sorted_df["Created Date"].dt.date
sorted_df["Created Hour"] = sorted_df["Created Date"].dt.hour
sorted_df["Created Weekday"] = sorted_df["Created Date"].dt.dayofweek

In [ ]:
daily_complaints = sorted_df.groupby("Created Day")["Created Day"] \
                            .count()
    
daily_complaints.plot.barh(figsize=(15,30));

In [ ]:
month_start = datetime(2019, 9, 2)
month_end = datetime(2019, 9, 30)

daily_complaints = sorted_df[(month_start <= sorted_df["Created Date"]) & (sorted_df["Created Date"] < month_end)] \
                            .groupby(['Created Day', 'Complaint Type'])['Created Day'] \
                            .count() \
                            .unstack('Complaint Type') \
                            .fillna(0)

ax = daily_complaints.plot(kind='bar', stacked=True, figsize=(30,10), rot=45)
ax.set_title("Noise complaints over the covered period")
ax.set_xlabel("Hour")
ax.set_ylabel("Count");

Seasonality is obvious, there is a weekly pattern, with a peak in noise complaints over the weekend. 

In [ ]:
hourly_complaints = sorted_df.groupby(['Created Hour', 'Complaint Type'])['Created Hour'].count().unstack('Complaint Type').fillna(0)
ax = hourly_complaints.plot(kind='bar', stacked=True, figsize=(20,10), rot=0)
ax.set_title("Aggregated noise complaints over the hours of a day")
ax.set_xlabel("Hour")
ax.set_ylabel("Count");

There is also a daily pattern, with lots of noise complaints during the night. We might want to look at the pattern we get when aggregating based on the hour and the weekday, to see if there are differences and validate our current approach (disregard weekday). We expect to have a similar pattern every day, but with different numbers. 

In [ ]:
hourly_complaints = sorted_df.groupby(['Created Weekday', 'Created Hour'])[['Unique Key']].count().reset_index()#.unstack('Complaint Type').fillna(0)  hourly_complaints
#ax = hourly_complaints.plot(kind='bar', stacked=True, figsize=(20,10), rot=0)
#ax.set_title("Aggregated noise complaints over the hours of a day")
#ax.set_xlabel("Hour")
#ax.set_ylabel("Count");

In [ ]:
hourly_complaints

In [ ]:
import seaborn as sns
sns.set()

g = sns.catplot(x='Created Hour', y='Unique Key', col='Created Weekday', data=hourly_complaints, kind='bar', col_wrap=2, sharey=False, height=4, aspect=2)
g = g.set_ylabels("Count")

Not really different shapes, there are just peaks on Friday and Saturday night that must be taken into account and have an effect on the last 3 plots. Question arises: Is it just party-related / nightly complaints that are responsible for the weekend peaks?

In [ ]:
g = sns.catplot(x='Created Weekday', y='Unique Key', col='Created Hour', data=hourly_complaints, kind='bar', col_wrap=2, sharey=False, height=4, aspect=2)
g = g.set_ylabels("Count")

Not only are there more nighttime noise-complaints on the Weekend, there are also more daytime complaints, probably because people tend to be at home over the weekend, where they are more likely to be disturbed than at work. This is something that should also be taken into account when **reasoning about other types of complaints**, i. e. when interpretating a number of complaints we should always think about factors like the weekday, the hour, etc. 

## Comment on the plots

What can be done with noise:
- Provide general method to analyse seasonality, that can be used for any column
- Compute daily mean, then spot outliers 
- Compute hourly mean (weekday), then spot outliers
- Look at noise distribution across NTAs, normalise with NTA population

Noise seems to have quite a high variance, CIs wouldn't give much insights.

## Analyse noise based on location

We want to identify noisy NTAs.

In [ ]:
sorted_df["NTA"] = np.nan
sorted_df.loc[sorted_df.Location.notnull(), "NTA"] = sorted_df.Location[sorted_df.Location.notnull()].apply(get_nta)

Load NTA population sizes:

In [ ]:
nta_pop_df = pd.read_csv(path_nta_population)
nta_pop_df = nta_pop_df[nta_pop_df.Year == 2010].reset_index()

In [ ]:
nta_complaints = sorted_df.groupby("NTA")[["Unique Key"]] \
                          .count() \
                          .rename(columns={"Unique Key": "Count"})

nta_complaints_extended = nta_complaints.merge(nta_pop_df.set_index("NTA Code"), left_index=True, right_index=True)
nta_complaints_extended["Normalized Count"] = nta_complaints_extended["Count"] / nta_complaints_extended["Population"]
nta_complaints_extended.loc[nta_complaints_extended.Population == 0, "Normalized Count"] = np.nan
nta_complaints_extended = nta_complaints_extended.reset_index().set_index("NTA Name").drop(columns=["level_0"])
nta_complaints_extended.head()

In [ ]:
nta_complaints_extended["NC Rank"] = nta_complaints_extended["Normalized Count"].rank(method="min", ascending=False)

In [ ]:
nta_complaints_extended[["Count", "Population", "NC Rank"]].sort_values(by="Count", ascending=False)[:20]

In [ ]:
nta_complaints_extended[["Normalized Count", "Population"]].sort_values(by="Normalized Count", ascending=False)[5:25]

In [ ]:
nta_complaints_extended[nta_complaints_extended.Population < 10000]

In [ ]:
nta_complaints_extended[["Count"]].sum().tolist()[0]

## Look at type of complaints in top 20 NTAs (based on normalized number of complaints)